In [1]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

### Preprocessing of sp-dep-jour-2023-06-30-16h26.csv 
##### First part of the dashboard

In [4]:
data = pd.read_csv('../raw_data/sp-dep-jour-2023-06-30-16h26.csv', sep=';')
data.head()

,dep,jour,pop,P,T,Ti,Tp,Td,cl_age90
0,01,13/05/2020,656955,9,340,"1,37","2,65","51,75",0
1,01,14/05/2020,656955,9,440,"1,37","2,05","66,98",0
2,01,15/05/2020,656955,5,454,"0,76","1,1","69,11",0
3,01,16/05/2020,656955,0,151,0,0,"22,98",0
4,01,17/05/2020,656955,0,32,0,0,"4,87",0


In [5]:
for name in ['Ti','Td','Tp']:
    data[name] = data[name].str.replace(',', '.')
    data[name] = data[name].astype(float)

In [6]:
df = data.copy()
df = df.fillna(0)
df['date'] = pd.to_datetime(data['jour'])
df.drop(['dep','jour','cl_age90'], axis=1, inplace=True)
df = df.groupby('date').agg({'P':'sum', 'T':'sum','Ti':'mean', 'Tp':'mean','Td':'mean', 'pop':'sum'}).reset_index()
df.head(2)

C:\Users\Guillaume\AppData\Local\Temp\ipykernel_13056\558924983.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(data['jour'])


,date,P,T,Ti,Tp,Td,pop
0,2020-05-13,889,39287,1.135490,2.275000,54.462255,67099037
1,2020-05-14,1001,42653,1.284902,2.294608,59.428333,67099037


In [7]:
df['P7'] = df['P'].rolling(window=7).mean()
df['T7'] = df['T'].rolling(window=7).mean()

In [9]:
df.to_csv('../preprocessed_data/epidemic_state.csv', index=False)